# The North Remembers Project

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import re
import random
from pprint import pprint
from time import sleep
import math
from itertools import groupby
from collections import OrderedDict as odict

### 取舍

数据存储用 github, 或者 bitbucket


提供导出为 rss, epub, pdf, github 自带检查历史的功能

不追求实效性



外链图片不作为重点
外链图片和视频链接, url 链接, 一并修复, 换行字符串等也一并处理, 七牛云?
md 中的 Latex 和 代码检测也在这里修复, pygment

### 调用关系





#### Collector

处理用户发出的命令

可以是 zhihu answer url, 问题页, 收藏夹页, 需要带有过滤条件
用户可能在过滤条件后再做修补删减

需要转为具体的爬虫任务, 并创建文件夹
单页收集到特定的文件夹, 否则收集到一个自动命名的文件夹

在文件夹下生成task

Resources
global_config
task_config (scanlist)


Collector.append(url, options)



#### Watcher



任务调度, 监视文件夹, 根据 task.yaml 抓取页面

task.yaml 也记录历史

task_type: single_page, 
update_batch: 10


track_new: true


通过扫描已有的待抓取任务, 在手动触发, 达到特定时间后, 分派抓取任务

    Watcher
        .watch()
        .remember()
        .status()
        .load_config_yaml
        .save_config_yaml
        .add_task add_tasks



Task

被调用抓取单一页面内容, 或者抓取一个 url 列表
抓取单一页面内容将存为 md, 不存 html

    Task
        ZhihuAnswerPageTask
        ZhihuAnswerIndexTask
        ZhihuColumnPageTask
        ZhihuColumnIndexTask

Task 会使用 Page.save / load

    .last_page()
    .fetch()
    .save() 传递一些参数给 page 如 watch_date
    .to_yaml_text()
    
    .priority
    .is_page_type
    .should_fetch
    .content_is_changed



ZhihuColumnPage
ZhihuColumnLister


Fetcher.request(url) => dict()? Page()

Page.write to / load from disk

PageTask.run => Fetcher.request(url) => New Page
PageTask.update => New Page.write
IndexTask.run => Fetcher.request(url) => [New Tasks]


Watcher -> PageTask -> Page (tmpl, fetcherAPI)
                            ZhihuColumnPage
                            ZhihuAnswerPage
                            WeixinArticlePage


Watcher -> IndexTask -> append more tasks
                        ZhihuColumnIndex
    





#### Fetcher

Page

页面内容分为 元数据 + 文章主体 + 评论, 评论先放放


Page.load()  返回一个读盘的页面

以md文件为单位

    ZhihuColumnPage
        .load(path)
        .write(path)
        .compare()

    attr
        task = ForeignKeyField(Task, related_name='task')
        watch_date = DateTimeField()
        title = CharField(null=True)
        author = CharField(null=True)
        content = TextField(null=True)
        metadata = TextField(null=True)
        topic = TextField(null=True)
        question = TextField(null=True)
        comment = TextField(null=True)





#### Cleaner

处理, 清理文本的规则



Exporter

根据 Template 转换为 rss, epub, pdf, 以及 github diff 预览

图片转存也算在 Template 里

In [ ]:
描述

以更加简洁清晰的排版阅读知乎. (这个需求有简悦)

用户指定一个知乎网站上的问题, zhihumark 抓取相应的回答内容, 以 markdown 文档的形式提供给用户阅读和下载. (提供 md 下载没用)





功能

可以自由定制想抓取的内容, 这包括: 抓取指定答主/收藏夹名/问题/话题下的回答, 抓取赞同数量超过N的回答.
可以设置黑名单, 不抓取特定的问题或者答主.
抓取评论, 以会话的形式组织起来, 让上下文相关的评论出现在一起, 尽量滤除没有价值的评论.
将收集到的一系列文档保存为epub或者pdf, 或者批量下载 markdown 文档, 以便离线阅读.
制作成 feed 以便放入 rss 阅读器, 获取实时更新.


观念

我们也不显示其他人搜索了什么关键字, 获取了什么页面, 下载了哪些电子书, 看到什么取决于你

我们相信评论是回答(或文章)可以分割的一部分, 许多评论很水, 有些很有价值.

相信评论区是对回答的重要补充, 也有随回答保留的必要, 但需处理评论区良莠不齐, 引用回复混乱的问题.



页面设计

单页面网站应用, 基本上类似搜索引擎的风格和排版.

输入框 - 接受输入ID或URL 答主/收藏夹名/问题名/话题名

选项 - 保留最前的n个回答?
选项 - 赞同数至少是?
选项 - 保留评论? 评论对话中至少有多少 likes
屏蔽 - 不感兴趣的话题, 问题, 答主
返回 - markdown文件列表

批量下载按钮

In [ ]:
import tools
tools.generate_ascii_title('flask')

### Test Collector

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collector import Collector

col = Collector(project_path='D:\DataStore\The North Remembers - Science')
# col = Collector(project_path='/project')
# topic_id = 19644231 # 古建筑
# topic_id = 19582176 # 建筑设计
# topic_id = 19573393 # 建筑史
# topic_id = 19568972 # 建筑学

In [ ]:
# url = 'https://zhuanlan.zhihu.com/wision'
# tip = '智堡研究所'
# # col.add(url, tip, lister_option=None, page_option=None)
# col.add(url, tip)
# str(col)

In [ ]:
urls = [
  'https://www.zhihu.com/topic/19577481/top-answers/', # 量子物理
  'https://www.zhihu.com/topic/19803067/top-answers/', # 量子通信
  'https://www.zhihu.com/topic/19938157/top-answers/', # 量子计算机
]

col.add_multiple(urls, lister_option={'zhihu_min_voteup': 1000})
str(col)

In [ ]:
import tools

t = tools.parse_type('https://www.zhihu.com/topic/19577481/top-answers/')

In [ ]:
t == tools.UrlType.ZhihuAnswerLister

In [ ]:
# collector.add(weixin_page_url)
# collector.add(zhihu_column_url)
# collector.add(zhihu_anwser_id=xxx)
# collector.add(zhihu_author_id=xxx)
# collector.add(column_id=xxx)

# collector.save(file=xxx)
# ...
# collector.rescan()
# collector.save(file=xxx)
# collector = Collector.load(file=xxx.yaml)

### Test Watcher

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from watcher import Watcher


# w = Watcher(r'D:\DataStore\The North Remembers - Social\篝火旁的故事')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\沉思的托克维尔')
# w = Watcher(r'D:\DataStore\The North Remembers - Science\何为现实？拉普拉斯之妖与薛定谔猫之决战')

# w = Watcher(r'D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学')

w = Watcher(r'D:\DataStore\test\机票')
# w = Watcher(r'D:\DataStore\The North Remembers - Game\旗舰评论——战略航空军元帅的旗舰')
# w = Watcher(r'D:\DataStore\test\经济')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\何瑫的写作工坊')
# w = Watcher(r'D:\DataStore\The North Remembers - Science/知乎话题 - 量子物理, 量子通讯, 量子计算机')


In [ ]:
for status in w.watch():
  print(status)

In [ ]:
from werkzeug.contrib.atom import AtomFeed
feed = AtomFeed('feed_name',
                  feed_url="feed_url.request.url", 
                  url="url.request.url_root")




In [ ]:
u = 'https://www.zhihu.com/question/19848582/answer/18231808'
question = parse_question(u)
answer = parse_answer(u)
# try:
#   title = question.title
#   detail = question.detail
# except GetDataErrorException:
#   # zhihu_oauth.exception.GetDataErrorException:
#   # A error happened when get data: 问题不存在
#   raise QuestionDeleteError(msg='问题已删除', url=url)


In [ ]:
question.title

In [ ]:
# 为什么 9·11 恐怖袭击中双子塔在一段时间的燃烧后倒塌了？
fetch_zhihu_answer('https://www.zhihu.com/question/19848582/answer/18231808')

In [ ]:
d = _
print(d.get('comments'))

### Test Generate Feed 

In [ ]:
from recorder import generate_feed

resp = generate_feed(r'D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学')
print(bytes.decode(resp[0]))

In [ ]:
resp = _

In [ ]:
resp.response

### Test Zhihu Fetch Error

In [ ]:
from fetcher_api.zhihu import fetch_zhihu_answer
from fetcher_api.zhihu import parse_answer
from fetcher_api.zhihu import parse_question

### Fetch comments MaxRetryError

In [ ]:
from pprint import pprint
from fetcher_api.zhihu import parse_question
from fetcher_api.zhihu import parse_answer
from fetcher_api.zhihu import zhihu_detect_with_client
from fetcher_api.zhihu import zhihu_detect
from fetcher_api.zhihu import get_comments_api_v4
from fetcher_api.zhihu import get_valuable_conversations_api_v4

### generate token

In [ ]:
from tools import generate_zhihu_token
generate_zhihu_token()

In [ ]:
from fetcher_api import zhihu

In [ ]:
client = zhihu.client

In [ ]:
c = client.collection(133030984)
int("133030984")

In [ ]:
for ans in c.answers:
  print(ans.author.name)

In [ ]:
from zhihu_oauth import ZhihuClient

TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)


In [ ]:
page = client.from_url('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
from fetcher_api.zhihu import fetch_zhihu_article

In [ ]:
fetch_zhihu_article('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
import os
os.path.basename('D:\Coding\TheNorthRemembers\page.py')

In [ ]:
os.path.__file__

In [ ]:
import wechatsogou

In [ ]:
ws_api = wechatsogou.WechatSogouAPI()

In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
list(ws_api.search_article('马前卒工作室'))

In [ ]:
topic_id = 11
f'generate_tip {topic_id}'

### Playground

In [ ]:
from pprint import pprint
from website.zhihu import zhihu_detect_with_client
from website.zhihu import zhihu_detect
from website.zhihu import get_comments_api_v4
from website.zhihu import get_valuable_conversations_api_v4
from website.zhihu import fetch_zhihu_answer
from website.zhihu import fetch_zhihu_article
import tools

In [ ]:
u = 'https://zhuanlan.zhihu.com/p/30722439'

fetch_zhihu_article(u)

In [ ]:
from markdown import markdown
from mdx_gfm import GithubFlavoredMarkdownExtension



def to_html(full_text, cut=0):
  ''' 转换为 html 用于输出 RSS 等
      必须有 data['full_text'] 才能 to_html() '''
  if cut and len(full_text) > cut:
    full_text = full_text[:cut] + f' ... (共 {len(full_text)} 字)'

  html = markdown(full_text, output_format='html5', extensions=[GithubFlavoredMarkdownExtension()])
  return html

In [ ]:
from feedgen.feed import FeedGenerator
fg = FeedGenerator()
fg.id('http://lernfunk.de/media/654321')
fg.title('Some Testfeed')
fg.author( {'name':'John Doe','email':'john@example.de'} )
fg.link( href='http://example.com', rel='alternate' )
fg.logo('http://ex.com/logo.jpg')
fg.subtitle('This is a cool feed!')
fg.link( href='http://larskiesow.de/test.atom', rel='self' )
fg.language('en')


fe = fg.add_entry()
fe.id('http://lernfunk.de/media/654321/1')
fe.title('The First Episode')
fe.link(href="http://lernfunk.de/feed")
fe.description('page.to_html(cut=0)' )


fe = fg.add_entry()
fe.id('http://lernfunk.de/media/654321/12222')
fe.title('The First Episode2222222')
fe.link(href="http://lernfunk.de/feed22222")
fe.description('3333333333333333' )

In [ ]:
fg.rss_file('feed.xml', pretty=True)

In [ ]:
print(bytes.decode(fg.rss_str(pretty=True)))